In [1]:
import os
import mlflow
from mlflow.tracking import MlflowClient

print('Tracking URI:', mlflow.get_tracking_uri())
print('MLFLOW_TRACKING_URI (env):', os.getenv('MLFLOW_TRACKING_URI'))

client = MlflowClient()  # nutzt automatisch MLFLOW_TRACKING_URI aus der Umgebung
experiments = client.search_experiments()
print('Experimente gefunden:', len(experiments))
for exp in experiments:
    print(f"- {exp.experiment_id}: {exp.name} (lifecycle_stage={exp.lifecycle_stage})")

Tracking URI: http://mlflow:5321
MLFLOW_TRACKING_URI (env): http://mlflow:5321
Experimente gefunden: 5
- 4: PoC Image Scoring 0.3 (lifecycle_stage=active)
- 3: PoC Image Scoring 0.2 (lifecycle_stage=active)
- 2: PoC Image Scoring 0.1 (lifecycle_stage=active)
- 1: API_Poc_0.2 (lifecycle_stage=active)
- 0: Default (lifecycle_stage=active)
